Use fancy machine learning to predict whether an article makes it into Nature/Science or PRL. This time we'll only look at articles in the physics.atom-ph section.

In [158]:
#Need to add parent directoy to sys.path to find 'metadataDB'
import sys
sys.path.append('../')

%matplotlib inline
import matplotlib.pyplot as plt 
import time
import numpy as np

# Natural language processing toolkit
# To use this, run nltk.download() and download 'stopwords'
from nltk.corpus import stopwords
s=stopwords.words('english') + ['']

# Machine learning
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline
from sklearn import metrics

# SQL
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from metadataDB.declareDatabase import *
from sqlalchemy import or_, and_

engine = create_engine("sqlite:///../arXiv_metadata.db", echo=False)
Base.metadata.bind = engine
DBsession = sessionmaker(bind=engine)
session = DBsession()

In [2]:
query = session.query(Article_Category)\
                    .join(Category)\
                    .join(Article)\
                    .filter(Category.name.like('%quant-ph'),
                            or_(Article.journal_ref.like('Physics Review Letters%'),
                                Article.journal_ref.like('Phys. Rev. Lett.%'),
                                Article.journal_ref.like('PRL%')))
abstractPRL = [x.article.abstract for x in query.all()]
titlePRL = [x.article.title for x in query.all()]

In [3]:
query = session.query(Article_Category)\
                    .join(Category)\
                    .join(Article)\
                    .filter(Category.name.like('%quant-ph'),
                            or_(Article.journal_ref.like('Nature%'),
                                Article.journal_ref.like('Nat.%'),
                                Article.journal_ref.like('Science%')))
abstractNatureScience = [x.article.abstract for x in query.all()]
titleNatureScience = [x.article.title for x in query.all()]

In [4]:
session.close_all()

In [5]:
# # Train with 80% of the data, test with 20%
# # First start with abstracts.

# indPRL = len(abstractPRL)*4/5
# indNatureScience = len(abstractNatureScience)*4/5

# train_abstract = abstractPRL[:indPRL] + abstractNatureScience[:indNatureScience]
# train_title = titlePRL[:indPRL] + titleNatureScience[:indNatureScience]
# train_target = [0]*indPRL + [1]*indNatureScience
# train_target_names = ['PRL']*indPRL + ['Nature']*indNatureScience

# test_abstract = abstractPRL[indPRL:] + abstractNatureScience[indNatureScience:]
# test_title = titlePRL[indPRL:] + titleNatureScience[indNatureScience:]
# test_target = [0]*len(abstractPRL[indPRL:]) + [1]*len(abstractNatureScience[indNatureScience:])
# test_target_names = ['PRL', 'Nature/Science']

In [80]:
# Train with 80% of the Nature data, test with 20% of the Nature data
# Choose the same number of PRL and Nature articles in the test sets.

indNatureScience = len(abstractNatureScience)*4/5
indPRL = len(abstractPRL) - (len(abstractNatureScience) - indNatureScience)

train_abstract = abstractPRL[:indPRL] + abstractNatureScience[:indNatureScience]
train_title = titlePRL[:indPRL] + titleNatureScience[:indNatureScience]
train_target = [0]*indPRL + [1]*indNatureScience
train_target_names = ['PRL']*indPRL + ['Nature']*indNatureScience

test_abstract = abstractPRL[indPRL:] + abstractNatureScience[indNatureScience:]
test_title = titlePRL[indPRL:] + titleNatureScience[indNatureScience:]
test_target = [0]*len(abstractPRL[indPRL:]) + [1]*len(abstractNatureScience[indNatureScience:])
test_target_names = ['PRL', 'Nature/Science']

In [223]:
print len(abstractNatureScience)
print len(abstractPRL)

693
2844


In [184]:
#SVC(kernel='linear') is good
text_abstract_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))),
                              ('tfidf', TfidfTransformer()),
                              ('clf', SVC(kernel='linear'))])
text_abstract_clf.fit(train_abstract, train_target)
predict_abstract = text_abstract_clf.predict(test_abstract)
print text_abstract_clf.predict(train_abstract)

[0 0 0 ..., 1 1 1]


In [219]:
text_title_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
                              ('tfidf', TfidfTransformer()),
                              ('clf', LinearSVC())])
text_title_clf.fit(train_title, train_target)
predict_title = text_title_clf.predict(test_title)
print text_abstract_clf.predict(train_title)

[0 0 0 ..., 0 0 0]


In [218]:
print predict_abstract
print predict_title

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1
 1 0 0 1 0 1 0 1 0 0 0 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 0 0 1 1 0 1 0 0 1 1 1
 1 1 1 1 1 1 0 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1
 1 1 1 0 1 0 0 1 1 0 0 1 1 0 1 0 0 1 1 0 1 0 0 0 1 1 0 0 1 1 1 1 0 1 0 0 0
 1 1 1 1 1 1 0 0 1 0 1 1 1 0 0 1 1 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0

In [209]:
#SVC(kernel='linear')
print(metrics.classification_report(test_target, predict_abstract,
                                    target_names=test_target_names))

                precision    recall  f1-score   support

           PRL       0.70      0.88      0.78       139
Nature/Science       0.83      0.62      0.71       139

   avg / total       0.77      0.75      0.74       278



In [212]:
#SVC(kernel='linear')
print(metrics.classification_report(test_target, predict_title,
                                    target_names=test_target_names))

                precision    recall  f1-score   support

           PRL       0.53      0.97      0.69       139
Nature/Science       0.84      0.15      0.26       139

   avg / total       0.69      0.56      0.47       278



In [224]:
def inverseVectorizer(val):
    return (key for key, value in text_abstract_clf.named_steps['vect'].vocabulary_.iteritems() if value == val).next()

# This is super inefficient!!!
sorted_coefs = sorted( ((i,v) for i, v in np.ndenumerate(text_abstract_clf.named_steps['clf'].coef_.todense()) ),
                      key=lambda x: x[1] )

print "Top 50 indicators of PRL:"
bottom = sorted_coefs[:50]
print ", ".join([ inverseVectorizer(item[0][1]) for item in bottom])
print ""
print "Top 50 indicators of Nature/Science:"
top = list(reversed(sorted_coefs[-50:]))
print ", ".join([ inverseVectorizer(item[0][1]) for item in top])

Top 50 indicators of PRL:
we propose, of the, propose, show, we, squeezing, method, show that, we show that, we show, resonance, scheme, rydberg, frequency, the spin, present, case, quantum light, discuss, consider, we study, entanglement entropy, we present, method for, prove, chain, generalized, chains, that the, shift, show that the, pm, ensemble, relations, between the, from the, macroscopic quantum, quantum dot, terms of, derive, analyze, in terms, in terms of, hidden, hand, cold, scheme for, phys, presence of, collective

Top 50 indicators of Nature/Science:
quantum, here, here we, and, of, to, been, have, here we demonstrate, in, these, has, information, photonic, fundamental, by, physics, here we report, systems, or, towards, as, however, circuits, demonstrate, integrated, devices, challenge, physical, new, communication, matter, technologies, many, single, demonstrated, we demonstrate, such, the, science, for, sensitivity, scalable, atoms, have been, annealing, at, mechanical,

In [222]:
print text_abstract_clf.predict([test_abstract[-4]])
print test_abstract[-4]

[1]
  To build a fault-tolerant quantum computer, it is necessary to implement a
quantum error correcting code. Such codes rely on the ability to extract
information about the quantum error syndrome while not destroying the quantum
information encoded in the system. Stabilizer codes are attractive solutions to
this problem, as they are analogous to classical linear codes, have simple and
easily computed encoding networks, and allow efficient syndrome extraction. In
these codes, syndrome extraction is performed via multi-qubit stabilizer
measurements, which are bit and phase parity checks up to local operations.
Previously, stabilizer codes have been realized in nuclei, trapped-ions, and
superconducting qubits. However these implementations lack the ability to
perform fault-tolerant syndrome extraction which continues to be a challenge
for all physical quantum computing systems. Here we experimentally demonstrate
a key step towards this problem by using a two-by-two lattice of
supercond